# Sequence generation for the pseudoRWM set of experiments

## Import statements and utilities

In [4]:
import numpy as np
import os
import matlab.engine
from alive_progress import alive_bar
import pandas as pd

In [5]:
def shuffle_along_axis(arr, axis):
    idx = np.random.rand(*arr.shape).argsort(axis=axis)
    return np.take_along_axis(arr,idx,axis=axis)

def shuffled(arr):
    arr_shuffled = arr.copy()
    np.random.shuffle(arr_shuffled)
    return(arr_shuffled)

def choose_n_and_delete(arr, N):
    chosen = np.random.choice(arr, size=N, replace=False)
    arr = np.delete(arr, np.where(np.isin(arr, chosen)))
    return chosen, arr

## Settings

In [6]:
exp_type = "pseudoRWMReps2"  # choose from ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]
num_conditions = 10 # number of different file sequences to generate (change if needed)
reps_dict = {"pseudoRWM": 12, "pseudoRWMCtrl": 12, "pseudoRWMConf": 11, "pseudoRWMReps": 11, "pseudoRWMConf2": 11, "pseudoRWMReps2": 11}  # number of repetitions after the first presentation
num_reps = reps_dict[exp_type] # number of stimulus repetitions (after first presentation; change if needed)
exact_reps =  True if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"] else False  # should the number of repetitions be exact or ok to exceed by 1?
use_matlab = False  # use matlab to call createstimsequence?

## Start MATLAB if needed

In [7]:
if use_matlab:
    m = matlab.engine.start_matlab()
    m.addpath(m.genpath("C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/"), nargout=0)
else:
    from numpy.matlib import repmat

## Main sequence creation function

In [20]:
class pseudoRWMSequenceMaker:

    def __init__(self, exp_type, num_reps, use_matlab=True, num_conditions=10, exact_reps=False, max_consec_goal_reps=24):

        # ================ SETTINGS ==========================================================================
        assert exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"], f"{exp_type} is not a valid exp_type"
        self.exp_type = exp_type
        self.num_reps = num_reps
        self.num_conditions = num_conditions
        self.use_matlab = use_matlab
        self.exact_reps = exact_reps
        self.to_dir = f"C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/{exp_type}/"
        self.has_group_row = self.exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]

        self.num_keys = 3
        self.max_stims = 6
        self.block_structure = np.array([6, 6, 6, 6, 6, 6]) # 6 blocks: 3*type (Points vs Goals)
        self.num_blocks = len(self.block_structure)
        self.trial_type_structure = np.arange(self.num_blocks)%2
        self.num_goals_blocks = self.num_blocks - np.sum(self.trial_type_structure)
        self.num_goals_trials = np.sum(self.block_structure[np.where(self.trial_type_structure==0)])*(self.num_reps+1)
        self.max_consec_goal_reps = max_consec_goal_reps

        # R: columns represent keyboard keys
        # each column says how many stimuli will be associated with that key
        # e.g., 1, 2, 3 means 1 stimulus will be associated with key 0, 2, with key 1, 3 with key 2, 
        # for a total of 6 items
        if self.exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf2", "pseudoRWMReps2"]:
            self.R = np.vstack((
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2])
            ))
        else:
            self.R = np.vstack((
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2])
            ))

        # goal images  
        self.all_fracts = np.arange(1, (self.num_goals_trials*2)+1)  # all fractal image numbers 
        self.all_ctrl_goal_sets = [[5, 17], [25, 26], [24, 30], [56, 59], [76, 77]]*2 # fractals to use in the Ctrl version includes goal and non-goal


    def _get_grouped_goal_sequences(self, all_images, trial_types, blocks, block_groups, block_seqprototypes):
        
        all_goal_images = all_images[0:len(all_images)//2] 
        all_nongoal_images = all_images[len(all_images)//2:]
        goal_images = np.array([])
        nongoal_images = np.array([])

        if self.exp_type == "pseudoRWMConf":
            n0, n1, n2 = 2, 2, 4   # for each block, we need 2 + 2 + 4 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMReps":
            n0, n1, n2 = 6, 10, 8  # for each block, we need 6 + 10 + 8 sets of goal/nongoal image pairs
        if self.exp_type == "pseudoRWMConf2":
            n0, n1, n2 = 1, 1, 2   # for each block, we need 1 + 1 + 2 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMReps2":
            n0, n1, n2 = 6, 18, 12  # for each block, we need 6 + 18 + 12 sets of goal/nongoal image pairs



        group0_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n0))
        group1_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n1))
        group2_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n2))

        group0_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n0))
        group1_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n1))
        group2_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n2))

        if self.exp_type == "pseudoRWMReps2":
            shared_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 1))
            shared_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 1))

        # block_goal_sequence_i is a dictionary for the current block where keys are stimuli IDs and values are sequences
        # of goal images for that stimulus
        for goal_block_i, block_i in enumerate(np.where(trial_types==0)[0]):
            ns = blocks[block_i]
            # dictionary for this block where keys are stimuli IDs and values are sequences of goal images for that stimulus
            block_goal_sequence_i = {}
            # dictionary for this block where keys are stimuli IDs and values are sequences of nongoal images for that stimulus
            block_nongoal_sequence_i = {}
            
            if self.exp_type == "pseudoRWMConf":
                # form goal and nongoal sequences for each group of the pseudoRWMConf condition
                group0_goal_sequences = np.repeat(group0_goal_images[goal_block_i], (self.num_reps+1)//2)
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4), 
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4)))
                group2_goal_sequences = np.repeat(group2_goal_images[goal_block_i], (self.num_reps+1)//4)
                
                group0_nongoal_sequences = np.repeat(group0_nongoal_images[goal_block_i], (self.num_reps+1)//2)
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4), 
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4)))
                group2_nongoal_sequences = np.repeat(group2_nongoal_images[goal_block_i], (self.num_reps+1)//4)

            elif self.exp_type == "pseudoRWMReps":
                # form goal and nongoal sequences for each group of the pseudoRWMReps condition
                one_fourth = (self.num_reps+1)//4  # 3 if there are 11(+1) reps
                one_sixth = (self.num_reps+1)//6   # 2 if there are 11(+1) reps
                
                group0_goal_sequences = np.hstack((
                    np.repeat(group0_goal_images[goal_block_i][0:one_fourth], one_fourth), group0_goal_images[goal_block_i][one_fourth:]))
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i][0], one_fourth), group1_goal_images[goal_block_i][1:]))
                group2_goal_sequences = np.hstack((
                    np.repeat(group2_goal_images[goal_block_i][0:one_sixth], one_fourth), group2_goal_images[goal_block_i][one_sixth:]))
                
                group0_nongoal_sequences = np.hstack((
                    np.repeat(group0_nongoal_images[goal_block_i][0:one_fourth], one_fourth), group0_nongoal_images[goal_block_i][one_fourth:]))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i][0], one_fourth), group1_nongoal_images[goal_block_i][1:]))
                group2_nongoal_sequences = np.hstack((
                    np.repeat(group2_nongoal_images[goal_block_i][0:one_sixth], one_fourth), group2_nongoal_images[goal_block_i][one_sixth:]))
                
            elif self.exp_type == "pseudoRWMConf2":
                # form goal and nongoal sequences for each group of the pseudoRWMConf condition
                group0_goal_sequences = np.repeat(group0_goal_images[goal_block_i], (self.num_reps+1))     # just one goal image
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//2), 
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//2)))                 # one goal image, used as a goal half the time and as a nongoal the rest of the time
                group2_goal_sequences = np.repeat(group2_goal_images[goal_block_i], (self.num_reps+1)//2)  # two goal images
                
                group0_nongoal_sequences = np.repeat(group0_nongoal_images[goal_block_i], (self.num_reps+1))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//2), 
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//2)))
                group2_nongoal_sequences = np.repeat(group2_nongoal_images[goal_block_i], (self.num_reps+1)//2)

            
            elif self.exp_type == "pseudoRWMReps2":
                # form goal and nongoal sequences for each group of the pseudoRWMReps condition
                # we want non-overlapping sequences, so they will end up being 2*(self.num_reps+1)
                # second_stim_order below needs to be edited accordingly
                three_quarters = 3*((self.num_reps+1)//4)  # 9 if there are 11(+1) reps
                half = (self.num_reps+1)//2                # 6 if there are 11(+1) reps
                one_quarter = (self.num_reps+1)//4         # 3 if there are 11(+1) reps
                
                group0_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], three_quarters), 
                    group0_goal_images[goal_block_i][:len(group0_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], three_quarters),
                    group0_goal_images[goal_block_i][len(group0_goal_images[goal_block_i])//2:]
                    ))
                group1_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], one_quarter), 
                    group1_goal_images[goal_block_i][:len(group1_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], one_quarter),
                    group1_goal_images[goal_block_i][len(group1_goal_images[goal_block_i])//2:]
                    ))
                group2_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], half), 
                    group2_goal_images[goal_block_i][:len(group2_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], half),
                    group2_goal_images[goal_block_i][len(group2_goal_images[goal_block_i])//2:]
                    ))
                
                group0_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], three_quarters), 
                    group0_nongoal_images[goal_block_i][:len(group0_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], three_quarters),
                    group0_nongoal_images[goal_block_i][len(group0_nongoal_images[goal_block_i])//2:]
                    ))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], one_quarter), 
                    group1_nongoal_images[goal_block_i][:len(group1_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], one_quarter),
                    group1_nongoal_images[goal_block_i][len(group1_nongoal_images[goal_block_i])//2:]
                    ))
                group2_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], half), 
                    group2_nongoal_images[goal_block_i][:len(group2_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], half),
                    group2_nongoal_images[goal_block_i][len(group2_nongoal_images[goal_block_i])//2:]
                    ))

                
            # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group
            first_stim_order = np.random.permutation(self.num_reps+1)
            second_stim_order = np.random.permutation(self.num_reps+1)
            if self.exp_type == "pseudoRWMReps2":  # stimulus sequences should not be overlapping
                second_stim_order += self.num_reps+1
            
            # get which stimulus is associated with which group 
            group_to_stim_dict = {}
            for key, value in block_groups[block_i].items():
                if value not in group_to_stim_dict:
                    group_to_stim_dict[value] = []
                group_to_stim_dict[value].append(key)
            # group_to_stim_dict[0][0] = the first stimulus of the first group, 
            # group_to_stim_dict[0][1] = the second stimulus of the first group, etc.
            # create a goal and a nongoal sequence for each block and stimulus
            for g_i, (group_goal_sequence, group_nongoal_sequence) in enumerate(zip(
                    [group0_goal_sequences, group1_goal_sequences, group2_goal_sequences], 
                    [group0_nongoal_sequences, group1_nongoal_sequences, group2_nongoal_sequences])):
                for stim_n, stim_order in enumerate([first_stim_order, second_stim_order]):
                    # goal sequences
                    block_goal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_goal_sequence[stim_order]
                    # nongoal_sequences
                    block_nongoal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_nongoal_sequence[stim_order]
            
            # create the full goal/nongoal sequences based on where stimuli are located
            tempgoalseq = np.empty((self.num_reps+1)*ns)
            tempnongoalseq = np.empty((self.num_reps+1)*ns)
            for stim_i in range(ns):
                tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]
            goal_images = np.hstack((goal_images, tempgoalseq))
            nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

        return goal_images, nongoal_images


    def make_sequences(self):
        with alive_bar(self.num_conditions, title=f"Making sequences", force_tty=True) as bar:
            for s_i in range(self.num_conditions):

                # checks to be passed to approve a sequence
                consec_goal_reps_check = False

                while not consec_goal_reps_check:
                    # ================ BLOCKS ===========================================================================
                    # blocks: array of length n with each element representing the block's set size
                    blocks = self.block_structure.copy()

                    # ================ TRIAL TYPES =======================================================================
                    # trial_types: array  where each element represent the sequence
                    # of trial types (1 = Points, 0 = Goals) a participant will experience
                    trial_types = self.trial_type_structure.copy()
                    for i in range(0, self.num_blocks//2, 2):
                        np.random.shuffle(trial_types[i:i+2]) 

                    # ================ RULES =============================================================================
                    # block_rules: a list where each element is a dictionary, with keys representing a stimulus image and
                    # values representing the solution for each stimulus
                    # here we shuffle within rows (only the first three columns), so that it's not always the same keys 
                    # that have 1, 2, or 3 stimuli associated with them (no difference for rows with 2, 2, 2) 
                    R_i = shuffle_along_axis(self.R, axis=1)  # mix up stim/action within the rule
                    block_rules = []
                    for block_i in range(self.num_blocks):
                        block_rules.append({i: k for i, k in enumerate([key_i for key_i in range(self.num_keys) for _ in range(R_i[block_i, key_i])])})

                    # ================ STIMULI ===========================================================================
                    # stim_sets: stimulus sets (folders from where images will be taken for each block)
                    stim_sets = (np.random.permutation(self.num_blocks)+1)
            
                    # block_stimuli: will contain dictionaries with an image number for each stimulus
                    block_stimuli = []
                    for block_i, ns in enumerate(blocks):
                        block_stimuli.append({i: s for i, s in enumerate((np.random.permutation(self.max_stims)+1)[0:ns])})

                    # block_seqprototypes: will contain dictionaries for each participant, with keys representing a set size and
                    # values as lists with a sequence of stimuli to be presented
                    # create a prototype (corresponding to stimuli rather than stimulus images) for each set size
                    block_seqprototypes = []
                    
                    # block_sequences: maps block_seqprototypes to corresponding stimulus sequences based on block_stimuli
                    block_sequences = []
                    
                    for block_i, ns in enumerate(blocks):
                        criterion_passed = False
                        if self.use_matlab:
                            while not criterion_passed:
                                temp_seqprototype = np.squeeze(np.array(m.createstimsequence(m.double(int(self.num_reps)), m.double(int(ns))))).astype(int)
                                # if we want the exact number of repetitions across stimuli, we need to pass this criterion before accepting the sequence
                                criterion_passed = np.all(np.unique(temp_seqprototype, return_counts=True)[1]==(self.num_reps+1)) or (not self.exact_reps)
                                # print(s_i, block_i, criterion_passed, np.unique(temp_seqprototype, return_counts=True)[1])
                            block_seqprototypes.append(temp_seqprototype)
                        else:
                            # worse (but faster) alternative if createstimsequence doesn't work
                            temp_seqprototype = []
                            for _ in range(self.num_reps+1):
                                temp_seqprototype = np.hstack((temp_seqprototype, (shuffled(np.arange(1, ns+1)))))
                            block_seqprototypes.append(temp_seqprototype)
                        # turn into stimuli (stimulus image number)
                        block_sequences.append(np.vectorize((block_stimuli[block_i]).get)(block_seqprototypes[block_i]-1))

                    # ================ GROUPS ===========================================================================
                    # only for some experiments, we divide stimuli into three groups that will receive different treatments 
                    # in terms of goal image presentation
                    # we need to ensure that stimuli in the same group have different correct keys
                    # because for these experiments each key is associated with two consecutive stimulus numbers, we can do
                    # this by making sure stimuli of the same group are associated with stimuli that are one number apart
                    # we also need to randomize which group is associated with the first, second, or third set of solutions
                    # block_groups: will contain dictionaries with a group number for each stimulus
                    block_groups = []
                    for block_i, ns in enumerate(blocks):
                        block_groups.append({i: g for i, g in enumerate(np.tile(np.random.permutation(ns//2), 2))})

                    # ================ GOAL IMAGES ======================================================================
                    all_images = shuffled(self.all_fracts)
                    if self.exp_type == "pseudoRWM":
                        goal_images = all_images[0:len(all_images)//2] 
                        nongoal_images = all_images[len(all_images)//2:]

                    elif self.exp_type == "pseudoRWMCtrl":
                        goal_set = self.all_ctrl_goal_sets[s_i]
                        # counterbalance which image is the goal and which one is the non-goal across participants
                        goal_images = np.repeat(goal_set[int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)
                        nongoal_images = np.repeat(goal_set[1-int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)

                    elif self.exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]:
                        goal_images, nongoal_images = self._get_grouped_goal_sequences(all_images, trial_types, blocks, block_groups, block_seqprototypes)
                    
                    # ================ CSV FILE ==========================================================================
                    # create csv
                    # rows: stim, correct key, set size, blocks, img_folders, img_nums, trial_type, goal_img, nongoal_img
                    goal_stim_count = 0

                    for block_i, ns in enumerate(blocks):
                        block_length = (self.num_reps + 1) * ns  # number of trials in a block
                        
                        this_block = np.full((9+self.has_group_row, block_length), np.nan)

                        _, unique_idx = np.unique(block_sequences[block_i], return_index=True)
                        block_cond = trial_types[block_i]
                        
                        this_block[0] = block_seqprototypes[block_i]  # stimulus number
                        this_block[1] = np.vectorize((block_rules[block_i]).get)(block_seqprototypes[block_i]-1) # correct key for the stimulus number
                        this_block[2] = np.repeat(ns, block_length)  # set size
                        this_block[3] = np.repeat(block_i+1, block_length)  # block number
                        this_block[4] = np.repeat(stim_sets[block_i], block_length)  # image folder
                        this_block[5] = block_sequences[block_i]  # stimulus number
                        this_block[6] = np.repeat(block_cond, block_length)  # trial type

                        if block_cond == 0:
                                this_block[7] = goal_images[goal_stim_count:goal_stim_count+block_length]
                                this_block[8] = nongoal_images[goal_stim_count:goal_stim_count+block_length]
                                goal_stim_count += block_length
                        
                        if self.has_group_row:
                            this_block[9] = np.vectorize((block_groups[block_i]).get)(block_seqprototypes[block_i]-1) # group for the stimulus ID

                        if block_i == 0:
                            train_seq = this_block
                            unique_stims = this_block[:, unique_idx]
                        else:
                            train_seq = np.column_stack((train_seq, this_block))
                            unique_stims = np.column_stack((unique_stims, this_block[:, unique_idx]))

                    # consec_goal_reps_check = True
                    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
                    if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]:
                        colnames += ["group"]
                    # print(np.sum(pd.DataFrame(train_seq.T, columns=colnames).query("trial_type == 0").goal_img.diff()==0))
                    consec_goal_reps_check = np.sum(pd.DataFrame(train_seq.T, columns=colnames).query("trial_type == 0").goal_img.diff()==0) <= self.max_consec_goal_reps

                # save output
                np.savetxt(f"{self.to_dir}seq{s_i+1}_learning.csv", train_seq, delimiter=",")
                # np.savetxt(f"{self.to_dir}seq{s_i+1}_learning_test.csv", train_seq, delimiter=",")
                bar()

        return 

## Create sequences

In [9]:
exp_type

'pseudoRWMReps2'

In [21]:
# seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=num_conditions, use_matlab=use_matlab, exact_reps=exact_reps)
seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=12, use_matlab=use_matlab, exact_reps=exact_reps, max_consec_goal_reps=48)
seqmkr.make_sequences()

Making sequences |                                        | ▁▃▅ 0/12 [0%] in 0s (0.0/s, eta: -) 

Making sequences |████████████████████████████████████████| 12/12 [100%] in 13.7s (0.88/s)                              


## Checks

In [22]:
# exp_type = "pseudoRWMConf2"

df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq1_learning.csv", header=None).T

colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]:
    colnames += ["group"]
df.columns = colnames
df["condition"] = np.where(df.goal_img.isna(), 1, 0)

In [23]:
df.head()

,stim,correct_key,set_size,block,img_folder,stim_img,trial_type,goal_img,nongoal_img,group,condition
0,1.0,0.0,6.0,1.0,2.0,5.0,0.0,315.0,33.0,0.0,0
1,2.0,1.0,6.0,1.0,2.0,2.0,0.0,352.0,146.0,1.0,0
2,3.0,1.0,6.0,1.0,2.0,1.0,0.0,315.0,33.0,2.0,0
3,5.0,2.0,6.0,1.0,2.0,6.0,0.0,315.0,33.0,1.0,0
4,4.0,1.0,6.0,1.0,2.0,3.0,0.0,315.0,33.0,0.0,0


In [24]:
# exp_type = "pseudoRWMConf"

for n in range(1, 11):
    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{n}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)

    temp_df = df.groupby(["block", "group", "stim"], as_index=False).correct_key.first()
    temp_df.rename(columns={"correct_key": "stim_corr_key"}, inplace=True)
    temp_df["other_stim_corr_key"] = temp_df.loc[np.tile([0, -2], int(np.ceil(len(temp_df)/2))) + np.arange(1, len(temp_df)+1), "stim_corr_key"].reset_index(drop=True)       
    print(n, temp_df.query("stim_corr_key == other_stim_corr_key").block.unique())

1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []


In [25]:
# exp_type = "pseudoRWMConf2"

for n in range(1, 11):
    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{n}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2"]:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)

    print(n, np.sum(df.query("condition == 0").goal_img.diff().fillna(999) == 0))

1 48
2 47
3 47
4 48
5 47
6 48
7 47
8 48
9 48
10 48


In [26]:
len(df.query("condition == 0"))

216

In [27]:
temp_df = df.groupby(["block", "group", "stim"], as_index=False).correct_key.first()
temp_df.rename(columns={"correct_key": "stim_corr_key"}, inplace=True)
temp_df["other_stim_corr_key"] = temp_df.loc[np.tile([0, -2], int(np.ceil(len(temp_df)/2))) + np.arange(1, len(temp_df)+1), "stim_corr_key"].reset_index(drop=True)       
temp_df.query("stim_corr_key == other_stim_corr_key")  

,block,group,stim,stim_corr_key,other_stim_corr_key


In [28]:
temp_df = df.query("condition == 0").groupby(["block", "group", "stim"]).correct_key.value_counts()
temp_df

block  group  stim  correct_key
1.0    0.0    2.0   1.0            12
              5.0   2.0            12
       1.0    3.0   1.0            12
              6.0   2.0            12
       2.0    1.0   0.0            12
              4.0   2.0            12
3.0    0.0    1.0   0.0            12
              4.0   1.0            12
       1.0    2.0   1.0            12
              5.0   2.0            12
       2.0    3.0   1.0            12
              6.0   2.0            12
5.0    0.0    2.0   1.0            12
              5.0   2.0            12
       1.0    1.0   0.0            12
              4.0   2.0            12
       2.0    3.0   1.0            12
              6.0   2.0            12
Name: correct_key, dtype: int64

In [29]:
df.groupby("block").trial_type.value_counts()

block  trial_type
1.0    0.0           72
2.0    1.0           72
3.0    0.0           72
4.0    1.0           72
5.0    0.0           72
6.0    1.0           72
Name: trial_type, dtype: int64

In [30]:
df.groupby(["block", "group"]).goal_img.unique()

block  group
1.0    0.0          [264.0, 393.0, 41.0, 90.0, 22.0, 427.0, 44.0]
       1.0      [264.0, 105.0, 205.0, 61.0, 340.0, 14.0, 394.0...
       2.0      [264.0, 327.0, 189.0, 407.0, 166.0, 363.0, 168...
2.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
3.0    0.0       [359.0, 184.0, 154.0, 266.0, 36.0, 262.0, 421.0]
       1.0      [256.0, 126.0, 351.0, 359.0, 424.0, 230.0, 342...
       2.0      [359.0, 130.0, 341.0, 378.0, 52.0, 389.0, 43.0...
4.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
5.0    0.0        [254.0, 204.0, 195.0, 32.0, 336.0, 37.0, 404.0]
       1.0      [310.0, 45.0, 259.0, 215.0, 114.0, 111.0, 175....
       2.0      [304.0, 141.0, 254.0, 408.0, 372.0, 281.0, 81..

In [31]:
df.groupby(["block", "group"]).nongoal_img.unique()

block  group
1.0    0.0        [18.0, 128.0, 88.0, 127.0, 119.0, 383.0, 106.0]
       1.0      [18.0, 143.0, 423.0, 369.0, 3.0, 24.0, 249.0, ...
       2.0      [18.0, 192.0, 66.0, 27.0, 96.0, 40.0, 373.0, 3...
2.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
3.0    0.0          [1.0, 99.0, 160.0, 15.0, 251.0, 201.0, 422.0]
       1.0      [73.0, 157.0, 176.0, 1.0, 388.0, 235.0, 348.0,...
       2.0      [1.0, 7.0, 131.0, 431.0, 80.0, 267.0, 307.0, 3...
4.0    0.0                                                  [nan]
       1.0                                                  [nan]
       2.0                                                  [nan]
5.0    0.0       [108.0, 278.0, 355.0, 413.0, 279.0, 430.0, 82.0]
       1.0      [190.0, 314.0, 253.0, 200.0, 9.0, 353.0, 284.0...
       2.0      [121.0, 240.0, 108.0, 197.0, 234.0, 332.0, 29..

In [32]:
df.groupby(["block"]).goal_img.apply(lambda x: len(np.unique(x)))

block
1.0    37
2.0     1
3.0    37
4.0     1
5.0    37
6.0     1
Name: goal_img, dtype: int64

In [36]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

block  group
1.0    0.0                                 [1, 1, 1, 1, 1, 1, 18]
       1.0      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
       2.0               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12]
2.0    0.0                                                   [24]
       1.0                                                   [24]
       2.0                                                   [24]
3.0    0.0                                 [1, 1, 1, 1, 1, 1, 18]
       1.0      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
       2.0               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12]
4.0    0.0                                                   [24]
       1.0                                                   [24]
       2.0                                                   [24]
5.0    0.0                                 [1, 1, 1, 1, 1, 1, 18]
       1.0      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
       2.0               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [34]:
df.groupby(["block", "stim"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

block  stim
1.0    1.0              [1, 1, 1, 1, 1, 1, 6]
       2.0                       [1, 1, 1, 9]
       3.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       4.0              [1, 1, 1, 1, 1, 1, 6]
       5.0                       [1, 1, 1, 9]
       6.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
2.0    1.0                               [12]
       2.0                               [12]
       3.0                               [12]
       4.0                               [12]
       5.0                               [12]
       6.0                               [12]
3.0    1.0                       [1, 1, 1, 9]
       2.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       3.0              [1, 1, 1, 1, 1, 1, 6]
       4.0                       [1, 1, 1, 9]
       5.0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
       6.0              [1, 1, 1, 1, 1, 1, 6]
4.0    1.0                               [12]
       2.0                               [12]
       3.0                               [12]
       4.0            

In [37]:
df.groupby(["block", "group", "stim"]).correct_key.first()

block  group  stim
1.0    0.0    2.0     1.0
              5.0     2.0
       1.0    3.0     1.0
              6.0     2.0
       2.0    1.0     0.0
              4.0     2.0
2.0    0.0    3.0     1.0
              6.0     2.0
       1.0    2.0     0.0
              5.0     2.0
       2.0    1.0     0.0
              4.0     1.0
3.0    0.0    1.0     0.0
              4.0     1.0
       1.0    2.0     1.0
              5.0     2.0
       2.0    3.0     1.0
              6.0     2.0
4.0    0.0    1.0     0.0
              4.0     1.0
       1.0    2.0     0.0
              5.0     2.0
       2.0    3.0     1.0
              6.0     2.0
5.0    0.0    2.0     1.0
              5.0     2.0
       1.0    1.0     0.0
              4.0     2.0
       2.0    3.0     1.0
              6.0     2.0
6.0    0.0    2.0     0.0
              5.0     2.0
       1.0    1.0     0.0
              4.0     1.0
       2.0    3.0     1.0
              6.0     2.0
Name: correct_key, dtype: float64

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

block  group
1.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
2.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
3.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
4.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
5.0    0.0                  [2, 2, 2, 6, 6, 6]
       1.0      [2, 2, 2, 2, 2, 2, 2, 2, 2, 6]
       2.0            [2, 2, 2, 2, 2, 2, 6, 6]
6.0    0.0                                [24]
       1.0                                [24]
       2.0                                [24]
Name: goal_img, dtype: object

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: len(np.unique(x)))

block  group
1.0    0.0       6
       1.0      10
       2.0       8
2.0    0.0       1
       1.0       1
       2.0       1
3.0    0.0       1
       1.0       1
       2.0       1
4.0    0.0       6
       1.0      10
       2.0       8
5.0    0.0       6
       1.0      10
       2.0       8
6.0    0.0       1
       1.0       1
       2.0       1
Name: goal_img, dtype: int64

In [ ]:
df.groupby(["block", "goal_img"]).stim.apply(lambda x: len(np.unique(x)))

block  goal_img
1.0    3.0         2
       15.0        2
       20.0        2
       22.0        2
       43.0        2
                  ..
5.0    336.0       2
       352.0       2
       353.0       2
       371.0       2
       418.0       2
Name: stim, Length: 72, dtype: int64

In [ ]:
# df.to_csv("test_seq.csv")

## Quit MATLAB if needed

In [ ]:
# quit matlab
if use_matlab:
    m.exit()